In [1]:
import numpy as np
from numpy import deg2rad, sqrt, pi, sin, cos
from numpy.linalg import norm
import matplotlib.pyplot as plt
from scipy.optimize import newton
from scipy.spatial.transform import Rotation

In [2]:
MU = 3.986004418E5
C = 300 # km/ms

# Exercício 1 (Parametrização das órbitas)

## Item a

Uma parametrização é o processo de encontrar equações paramétricas que descrevem uma curva ou superfície. Expressar essas formas em termos de parâmetros.

Uma curva é a imagem de uma função $\gamma: I\subseteq \mathbb{R}^n \to \mathbb{R}^m$.

Nesse primeiro caso, quero encontrar a parametrização cartesiana de uma curva

$$\gamma(t) = \left(x(t), y(t)\right)$$

Da definição de coordenadas polares,

$$\begin{cases}
x(t) = r(t)\cos(\theta(t))\\
y(t) = r(t)\sin(\theta(t))\end{cases}$$

Sendo que $r(t)$ é a distância entre a origem e o ponto da curva e $\theta(t)$ é o ângulo marcado em sentido antihorário com a parte positiva do eixo $x$. Portanto, $r(t)$ é a mesma distância dada e $\theta(t) = \nu(t)$

$$\begin{cases}
x(t) = r(t)\cos(\nu(t))\\
y(t) = r(t)\sin(\nu(t))\end{cases}$$

Em seguida, definido o círculo da figura, igualando os comprimentos na horizontal:

$$a\cos E = ae + r\cos \nu$$

![](imgs/circulo_ecentrico.svg)

[Orbital Mechanics, Bryan Weber](https://orbital-mechanics.space/time-since-periapsis-and-keplers-equation/elliptical-orbits.html#eccentric-anomaly)

Sabendo que $ae$ é a distância entre o centro do círculo e a posição do planeta (distância do centro ao foco). 

---

**Demonstração** 

Uma característica da Elipse é que *soma das distâncias de um ponto para cada um de seus focos é constante*. Assim, quando o ponto está no eixo $x$, podemos mostrar que essa constante é igual a $2a$. Além disso, quando o ponto está a $90^\circ$ graus do eixo $x$ positivo, as duas distância são iguais e, portanto, valem $a$. Nesse esquema, denotando como distância do centro ao foco como $c$, pode ser obtida a relação

$$\text{(dist. centro ao foco)} = c = \sqrt{a^2-b^2}$$

A excentricidade é a razão entre a distância do centro até o foco e o semieixo maior.

$$e:= \frac{c}{a} = \sqrt{1 - \frac{b^2}{a^2}}$$

Portanto, dada a excentricidade $e$ e o semieixo maior $a$ a distância do centro ao foco é

$$c = ae$$

---

A igualdade dos comprimentos acima, indica que

$$x = r\cos\nu = a(\cos E - e)$$

De maneira similar, a coordenada y na elipse é dada por $y = b\sin E$. Da definição de excentricidade, $b = a \sqrt{1-e^2}$

$$y = a\sqrt{1-e^2}\sin E$$

## Item b

Da segunda Lei de Kepler,

$$\Delta A = \frac{h}{2}\Delta t$$

Para a área total, teremos o período

$$\pi ab = \frac{h}{2}T \Leftrightarrow T = 2\pi\sqrt{\frac{a^3}{\mu}}$$

É uma equação que envolve uma função transcendente e não tem solução analítica (sem solução algébrica para $E$). Faz uso de métodos iterativos ou expansão em série.

Vide: https://en.wikipedia.org/wiki/Kepler's_equation#Equation

In [3]:
def perifocalPos(tp, a, e, mu = MU):
    
    def kepler(E, M, e):
        # Equação de Kepler na forma: f(E) = 0
        return E - e * sin(E) - M

    def dKepler(E, M, e):
        # Derivada da Equação de Kepler f'(E)
        return 1 - e * cos(E)

    tp = np.atleast_1d(tp)
    
    T = 2 * pi / sqrt(mu) * a**(3 / 2) # Período Orbital (seg)
    M = (2*pi/T * tp) % (2*pi) # Anomalia Média

    # Resolve a Equação de Kepler para a Anomalia Excêntrica
    E0 = M
    E = newton(func=kepler, x0=E0, fprime=dKepler, args=(M, e))
    
    # Resposta do item a)
    x = a*cos(E) - a*e
    y = a*sqrt(1-e**2)*sin(E)
    
    return np.array([x, y])

## Item c

https://orbital-mechanics.space/classical-orbital-elements/orbital-elements-and-the-state-vector.html

In [4]:
def posVec(tp, a, e, w, i, o, mu = MU):
    pPos = perifocalPos(tp, a, e, mu)
    
    perifocal3D = np.vstack([pPos, np.zeros(pPos.shape[1])])
    
    # Realiza as três rotações
    angles = -np.array([w, i, o]).T
    angles = deg2rad(angles)
    R = Rotation.from_euler("ZXZ", angles).as_matrix()
    pos = R.T @ perifocal3D

    return pos

## Teste

In [5]:
# Dados
a = np.array([15300, 16100, 17800, 16400]) # km
e = np.array([.41, .342, .235, .3725])
w = np.array([60, 10, 30, 60]) # deg
i = np.array([30, 30, 0, 20]) # deg
o = np.array([0, 40, 40, 40]) # deg
tp = np.array([4708.5603, 5082.6453, 5908.5511, 5225.3666])
TOA = np.array([60000] * 4)
TOT = np.array([13581.1080927 , 19719.32768037, 11757.73393255, 20172.46081236])

In [6]:
satPos = []
for k in range(4):
    pos = posVec(tp[k], a[k], e[k], w[k], i[k], o[k])
    satPos.append(pos)

satPos = np.array(satPos).reshape([4, 3])
satPos

array([[-17198.94636766,  -3357.8884269 ,  -1938.67778718],
       [-16764.51326576,   -188.27453647,   6138.26955927],
       [-18646.04514963,  -1962.47472564,      0.        ],
       [-12159.76207073, -13896.76819502,  -1029.81652816]])

# Exercício 2

## Item a


$$TOF = TOA - TOT$$

$$\rho = c \cdot TOF$$

$$c = 3\times 10^8\ \text{m/s}$$

In [7]:
TOF = TOA - TOT
TOF

array([46418.8919073 , 40280.67231963, 48242.26606745, 39827.53918764])

In [8]:
rho = TOF*C/1000
rho

array([13925.66757219, 12084.20169589, 14472.67982024, 11948.26175629])

## Item b

Sistema não linear, pois as equações não são combinações lineares das variáveis $(x,y,z)$

$$ (x - x)^2 + (y - y_i)^2 + (z - z_i)^2 = \rho_i^2$$

## Item c e d

$$J(x,y,z) = \frac{1}{2}\sum_i\left(\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - \rho_i \right)^2$$

$J$ é zero quando acertamos a posição do receptor. Para qualquer outro valor, o valor de $J$ será maior. Dessa forma, a posição do drone é um mínimo global de $J$.

$$\partial_x J = \sum_i(x-x_i)\frac{\sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2} - \rho_i}{ \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2}}$$

Para encurtar,

$$\hat \rho_i := \sqrt{(x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2}$$

$$\partial_x J = \sum_i(x-x_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$

Por simetria, obtém-se as outras derivadas que compõem o gradiente

$$\partial_y J = \sum_i(y-y_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$
$$\partial_z J = \sum_i(z-z_i)\frac{\hat \rho_i - \rho_i}{\hat \rho_i}$$

$$\nabla J = \begin{bmatrix}
    \partial_x J \\
    \partial_y J \\
    \partial_z J \\
\end{bmatrix}$$

Em formato de matriz, defino

$$\mathbf{x} = (x, y, z)$$

$$\mathbf{R} = \begin{bmatrix}
    x_1 & y_1 & z_1 \\
    x_2 & y_2 & z_2 \\
    & \vdots & \\
    x_n & y_n & z_n \\
\end{bmatrix}$$

$$\mathbf{d} = \mathbf{x} - \mathbf{R} = \begin{bmatrix}
    x - x_1 & y - y_1 & z - z_1 \\
    x - x_2 & y - y_2 & z - z_2 \\
    & \vdots & \\
    x - x_n & y - y_n & z - z_n \\
\end{bmatrix}_{(n\times 3)}$$

$$\boldsymbol{\rho} = \begin{bmatrix}
    \rho_1 \\ \rho_2 \\ \vdots \\ \rho_n
\end{bmatrix},\quad
\boldsymbol{\hat \rho} = \begin{bmatrix}
    \hat\rho_1 \\ \hat\rho_2 \\ \vdots \\ \hat\rho_n
\end{bmatrix},\quad
\frac{\boldsymbol{\hat \rho}  - \boldsymbol{\rho}}{\boldsymbol{\hat \rho} } = \begin{bmatrix}
    (\hat \rho_1 - \rho_1)/\hat \rho_1 \\ 
    (\hat \rho_2 - \rho_2)/\hat \rho_2 \\ 
    \vdots \\ 
    (\hat \rho_n - \rho_n)/\hat \rho_n
\end{bmatrix}_{(n\times 1)}$$

Logo, o vetor gradiente é

$$\nabla J = \mathbf{d}^T \left(\frac{\boldsymbol{\hat \rho}  - \boldsymbol{\rho}}{\boldsymbol{\hat \rho} }\right) = \mathbf{d}^T \left(1 - \frac{\boldsymbol{\rho}}{\boldsymbol{\hat \rho} }\right) $$

In [9]:
def grad(x):
    # satPos, rho são variáveis globais
    d = x - satPos
    rhoHat = norm(d, axis=1)
    return d.T @ (1 - rho/rhoHat)

In [13]:
grad(np.array([-6420., -6432., 6325.]))

array([-7.98092184e-10,  2.10875740e-10, -1.95422840e-10])

In [11]:
def gradient_descent(grad, x0, Nsteps=200, learningRate=0.6, momentum=0.7):
    x = x0
    vel = np.array([[0, 0, 0]], dtype='float64')
    
    for i in range(Nsteps):
        g = grad(x)
        
        vel = momentum * vel - learningRate * g
        x += vel
    
    return x

In [12]:
x0 = np.array([[0, 0, 0]], dtype='float64')

gradient_descent(grad, x0, Nsteps=200)

array([[-6420., -6432.,  6325.]])